# SPACESHIP TITANIC
**EXPLORATORY DATA ANALYSIS**

## Índice

1. Lectura del archivo y descripción del dataset
2. Selección de los datos de interés a analizar 
3. Limpieza de los datos.

    3. Elementos nulos o vacíos
    3. Valores extremos

4. Análisis exploratorio

    4. Análisis descriptivo (gráficos y tal)
    4. Análisis estadístico
    
        - Comprobación de la normalidad y homogeneidad de la varianza
        - Aplicación de pruebas estadísticas (pone que al menos 3)

5. Conclusiones





